In [1]:
import os
import shutil
import json
import datetime
import math
import numpy as np
import pandas as pd
from time import time, strftime
from collections import Counter
from keras import backend as K
from keras.models import load_model, Sequential, Model
from keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, BatchNormalization
from keras.layers import Dropout, Bidirectional, Flatten, Input, Reshape
from keras.layers.merge import Concatenate, Add, concatenate, add

def s_c(x):
    return [x[:, i:i+1] for i in range(len(x[0]))]

def save_preds(preds, cb=False):
    preds = np.ravel(preds)
    assert len(preds)==338489
    avg = np.average(preds)
    std = np.std(preds)
    p = '../%s%s_tl_result_%.4f_%.4f_%s.csv' % ('cb_' if cb else '', 
        strftime('%H%M_%m%d'), avg, std, 'args.m')

    df = pd.DataFrame({'instanceID': te_df_['instanceID'].values, 'proba': preds})
    df.sort_values('instanceID', inplace=True)
    df.to_csv(p, index=False)

    if cb: 
        return avg, std, p
    else:
        print('\nTrain average: ', tr_avg)
        print('Preds average: ', avg)
        print('Preds std dev.: ', std)
        print('\nWritten to: ', p)


class predCallback(Callback):
    def __init__(self, test_data):
        self.te_x = test_data

    def on_epoch_end(self, epoch, logs={}):
        predict_probas = np.ravel(self.model.predict(self.te_x, batch_size=40960, verbose=1))
        avg, std, p = save_preds(predict_probas, cb=True)
        print('\nTr avg: %.4f,   avg: %.4f, std: %.4f, written to: %s\n'%(tr_avg, avg, std, p))

Using TensorFlow backend.


In [3]:
features = ['appID', 'connectionType', 'age', 'telecomsOperator', 'gender', 'education', 'clickTime_h', 'weekDay',
            'marriageStatus', 'appPlatform', 'clickTime_m']
features = features[:11]

tr_ui_ = pd.read_csv('../data/pre/new_tr_ui.csv', header=None)
te_ui = pd.read_csv('../data/pre/new_te_ui.csv', header=None).values
tr_ua_ = pd.read_csv('../data/pre/new_tr_ua.csv', header=None)
te_ua = pd.read_csv('../data/pre/new_te_ua.csv', header=None).values
tr_adAppCate_ = pd.read_csv('../data/pre/new_adAppCate_tr.csv', index_col=0)
te_adAppCate = pd.read_csv('../data/pre/new_adAppCate_te.csv', index_col=0).values
tr_df_ = pd.read_csv('../data/pre/new_generated_train.csv')
te_df_ = pd.read_csv('../data/pre/new_generated_test.csv')

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
bs = 4096
print('\n\nSeed:', 62, 'Batch size: ', bs)
va_ui = tr_ui_.sample(frac=.1, random_state=62)
tr_ui = tr_ui_.drop(va_ui.index, axis=0).values
va_ui = va_ui.values

va_ua = tr_ua_.sample(frac=.1, random_state=62)
tr_ua = tr_ua_.drop(va_ua.index, axis=0).values
va_ua = va_ua.values


va_adAppCate = tr_adAppCate_.sample(frac=.1, random_state=62)
tr_adAppCate = tr_adAppCate_.drop(va_adAppCate.index, axis=0).values
va_adAppCate = va_adAppCate.values

# va_df = tr_df.loc[tr_df['clickTime_d'] == 24]
va_df = tr_df_.sample(frac=.1, random_state=62)
tr_df = tr_df_.drop(va_df.index, axis=0)

tr_df = tr_df[features+['label']]
va_df = va_df[features+['label']]
te_df = te_df_[features]

tr = tr_df.values
va = va_df.values
te_x = te_df.values
np.random.shuffle(tr)
input_length = len(tr[0])-1
max_feature = max(tr.max(), va.max() , te_x.max())+1
max_feature_ui = max(tr_ui.max(), te_ui.max(), va_ui.max())+1
max_feature_ua = max(tr_ua.max(), te_ua.max(), va_ua.max())+1
max_f_cols = tr_df.max().values[:-1]+1
print('--- Train cols: ', tr_df.columns)
print('--- Max feature:', max_feature, max_feature_ua,max_feature_ui)
print('--- Max column features:', max_f_cols)

tr_x, tr_y = tr[:, :-1], tr[:, -1:]
va_x, va_y = va[:, :-1], va[:, -1:]
tr_avg = np.average(tr_y)



Seed: 62 Batch size:  4096
--- Train cols:  Index(['appID', 'connectionType', 'age', 'telecomsOperator', 'gender',
       'education', 'clickTime_h', 'weekDay', 'marriageStatus', 'appPlatform',
       'clickTime_m', 'label'],
      dtype='object')
--- Max feature: 473 222 210
--- Max column features: [473   5  81   4   3   8  24   8   4   3  60]


In [14]:
te_ua_ = pd.read_csv('../data/pre/new_te_ua.csv', header=None)

In [25]:
te_df_.loc[te_df_.userID==2707361]

,Unnamed: 0,instanceID,label,creativeID,userID,positionID,connectionType,telecomsOperator,adID,camgaignID,...,hometown,residence,sitesetID,positionType,appCategory,clickTime_d,weekDay,clickTime_h,clickTime_m,conversionTime_d
111040,111040,111041,-1,1456,2707361,2334,1,3,3379,411,...,909,909,0,1,209,31,4,12,30,0


In [27]:
ua = pd.read_csv('../data/pre/user_app_actions.csv')
ua.loc[ua.userID==2707361]

,userID,installTime,appID
5792686,2707361,11031,232223
5792687,2707361,11040,425185
5792688,2707361,11042,427052
5792689,2707361,11046,139926
5792690,2707361,11049,95771
5792691,2707361,11054,140881
5792692,2707361,11057,427189
5792693,2707361,11101,5991
5792694,2707361,11108,426896
5792695,2707361,11110,178664


In [7]:
optimizer = 'rmsprop' # rmsprop, adam
loss = 'binary_crossentropy'
metrics = ['binary_crossentropy'] # can't be empty in this script
workers = 1
tbCallBack = TensorBoard(log_dir='../meta/tbGraph/', histogram_freq=0,
             write_graph=True, write_images=True)
checkpoint = ModelCheckpoint('../trained_models/{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss', 
             verbose=1, save_best_only=True, period=1)
np.random.seed(7)

In [8]:
print('Building MLP >>>>>>>>>>>')
inp_x = Input(shape=(input_length, ))
inp_adCate = Input(shape=(28, ))
inp_ui = Input(shape=(28, ))
inp_ua = Input(shape=(28, ))

o_x = Embedding(max_feature, 64)(inp_x)
o_adCate = Embedding(2, 64)(inp_adCate)
o_ui = Embedding(max_feature_ui, 64)(inp_ui)
o_ua = Embedding(max_feature_ua, 64)(inp_ua)

o_x = Flatten()(o_x) # 16* max_feature
o_adCate = Flatten()(o_adCate) # 16* max_feature
o_ui = Flatten()(o_ui) # 16* max_feature
o_ua = Flatten()(o_ua) # 16* max_feature

y = concatenate([o_x, o_adCate, o_ui, o_ua])

y = Dense(1024, activation='relu')(y)
# y = Dense(1024, activation='tanh', kernel_regularizer='l1')(y)
y = Dense(512, activation='tanh')(y)
y = Dense(1, activation='sigmoid')(y)   

model = Model([inp_x, inp_adCate, inp_ui, inp_ua], y)

tr_x = [tr_x, tr_adAppCate, tr_ui, tr_ua]
te_x = [te_x, te_adAppCate, te_ui, te_ua]
va_x = [va_x, va_adAppCate, va_ui, va_ua]        

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.summary() 


Building MLP >>>>>>>>>>>
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 11)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 28)            0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 28)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 28)            0                                            
__________________________________________________________________

In [ ]:
model.fit(tr_x, tr_y, epochs=args.e, validation_data=(va_x, va_y), 
           shuffle=True, verbose=args.v, callbacks=[predCallback(te_x)] ,
        batch_size=bs)

In [ ]:
predict_probas = np.ravel(model.predict(te_x, batch_size=4096*2, verbose=args.v))
    save_preds(predict_probas)
